In [85]:
import pandas as pd


# Housing Prices dataset - Columns 'Quarter', 'Value' where 'Type of Employee' = "All employees" and 'Economic Sector NACE Rev 2' = "Construction (F)"
#df_housingPrices = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/HPA09/CSV/1.0/en')
#df_rentPrices = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/RIA02/CSV/1.0/en')
#df_GDP = pd.read_csv('/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_5454986.csv')

# Labour Costs dataset - Columns 'Quarter', 'Value' where 'Type of Employee' = "All employees" and 'Economic Sector NACE Rev 2' = "Construction (F)"
df_labourCosts = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/EHQ03/CSV/1.0/en')

# Construction Workers dataset - Columns 'Quarter', 'Value' where 'Sex' = "Both Sexes"
#df_constructionWorkers = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/QLF03/CSV/1.0/en')

# Buildings Built dataset - Columns 'Year' and 'Values'
#df_buildingsBuilt = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/NDA02/CSV/1.0/en')

#df_buildingsBuiltEircodes = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/NDA01/CSV/1.0/en')
#df_immigration = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/PEA15/CSV/1.0/en')
#df_wages = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/HPA09/CSV/1.0/en')
#df_population = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/HPA09/CSV/1.0/en')
#print(df_buildingsBuiltEircodes)

<ipython-input-85-b8f3a39a8508>:10: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_labourCosts = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/EHQ03/CSV/1.0/en')


In [87]:
new_df = pd.DataFrame()
import re

"""
filtered_housingPrices = df_housingPrices[(df_housingPrices["Type of Dwelling"] == 'All Dwelling Types') 
                    & (df_housingPrices["Dwelling Status"] == 'All Dwelling Statuses') 
                    & (df_housingPrices["Sectoral Flow"] == 'All sectoral flow types') 
                    & (df_housingPrices["Type of Sale"] == 'All Sale Types')
                    & (df_housingPrices["Statistic Label"] == 'Mean Sale Price')]

df2_mean = filtered_housingPrices.groupby('Year').mean().reset_index()
print(df2_mean)
"""

"""

dff_rentPrices = df_rentPrices[(df_rentPrices["Number of Bedrooms"] ==  'All bedrooms')
                                    & (df_rentPrices["Property Type"] ==  'All property types')]
dff_rentPrices['Location'] =  dff_rentPrices['Location'].str.replace(r'\b( Town| City)\b', '', regex=True)
dff_rentPrices['Location'] =  dff_rentPrices['Location'].str.split(', ').str[-1]
dff_rentPrices['Location'] =  dff_rentPrices['Location'].str.replace(' ', '', case=False)
dff_rentPrices['Location'] =  dff_rentPrices['Location'].str.split(',', n=1).str[0]
dff_rentPrices.loc[dff_rentPrices['Location'].str.contains('Dublin', case=False), 'Location'] = 'Dublin'
df2_mean = dff_rentPrices.groupby(['Year'])['VALUE'].mean().reset_index()
df2_mean
"""
dff_labourCosts = df_labourCosts[(df_labourCosts["Economic Sector NACE Rev 2"] == 'Construction (F)')
                                  & (df_labourCosts["Economic Sector NACE Rev 2"] == 'Construction (F)')]
dff_labourCosts


#dff_rentPrices
#print(dff_rentPrices['Location'].unique())
#updated_locations = [location.split(", ")[-1] for location in locations]

#filtered_rentPrices = df_rentPrices[(df_rentPrices["Number of Bedrooms"] ==  'All bedrooms')
#                                    & (df_rentPrices["Property Type"] ==  'All property types')]

#filtered_rentPrices
#df2_mean = filtered_housingPrices.groupby('Year').mean().reset_index()
#df2_mean
#df_housingPrices
#new_df = pd.merge(df_buildingsBuilt)







,STATISTIC,Statistic Label,TLIST(Q1),Quarter,C02665V03225,Economic Sector NACE Rev 2,C02397V02888,Type of Employee,UNIT,VALUE
12,EHQ03C01,Employment,20081,2008Q1,F,Construction (F),-,All employees,Number,146700
13,EHQ03C01,Employment,20081,2008Q1,F,Construction (F),1,"Managers, professionals and associated profess...",Number,24200
14,EHQ03C01,Employment,20081,2008Q1,F,Construction (F),2,"Clerical, sales and service employees",Number,16500
15,EHQ03C01,Employment,20081,2008Q1,F,Construction (F),3,"Production, transport, craft and other manual ...",Number,106100
96,EHQ03C01,Employment,20082,2008Q2,F,Construction (F),-,All employees,Number,144100
...,...,...,...,...,...,...,...,...,...,...
85527,EHQ03S9,Average Hourly Benefit in Kind (Seasonally Adj...,20223,2022Q3,F,Construction (F),3,"Production, transport, craft and other manual ...",Euro,NaN
85608,EHQ03S9,Average Hourly Benefit in Kind (Seasonally Adj...,20224,2022Q4,F,Construction (F),-,All employees,Euro,NaN
85609,EHQ03S9,Average Hourly Benefit in Kind (Seasonally Adj...,20224,2022Q4,F,Construction (F),1,"Managers, professionals and associated profess...",Euro,NaN
85610,EHQ03S9,Average Hourly Benefit in Kind (Seasonally Adj...,20224,2022Q4,F,Construction (F),2,"Clerical, sales and service employees",Euro,NaN


In [4]:
## Housing Prices
#TO DO - existem 1610 valores NULL na coluna VALUE. Encontrar a melhor forma de preencher este valores!! A1
# Soma dos valores null
#df_housingPrices[df_housingPrices["Type of Dwelling"].str.contains("All Dwelling Types").loc[:, ["Year", "VALUE"]]]
#df_labourCosts[df_labourCosts["Type of Employee"].str.contains("All employees")].loc[:, ["Quarter", "VALUE"]].isnull().sum()
"""
df_housingPrices[(df_housingPrices["Type of Dwelling"] == 'House') 
                    & (df_housingPrices["Dwelling Status"] == 'Existing') 
                    & (df_housingPrices["Sectoral Flow"] == 'All sectoral flow types') 
                    & (df_housingPrices["Type of Sale"] == 'All Sale Types')
                    & (df_housingPrices["Statistic Label"] == 'Mean Sale Price')]
"""
#df_housingPrices.columns
#df_housingPrices

## Labour Costs
#TO DO - existem 1610 valores NULL na coluna VALUE. Encontrar a melhor forma de preencher este valores!! A1
# Soma dos valores null
#df_labourCosts[df_labourCosts["Type of Employee"].str.contains("All employees")].loc[:, ["Quarter", "VALUE"]].isnull().sum()
#df_labourCosts


"""
locations = df_rentPrices["Location"].unique()
updated_locations = [location.split(", ")[-1] for location in locations]

# Print the updated location names
for location in updated_locations:
    print(location)
"""
df_immigration
#df_rentPrices[(df_rentPrices["Number of Bedrooms"] ==  'All bedrooms')
#              & (df_rentPrices["Property Type"] ==  'All property types')]

## Construction Workers
#TO DO - existem 307 valores NULL na coluna VALUE. Encontrar a melhor forma de preencher este valores!! A1
# Soma dos valores null
#df_constructionWorkers[df_constructionWorkers["Sex"].str.contains("Both sexes")].loc[:, ["Quarter", "VALUE"]].isnull().sum()

#df_constructionWorkers
#df_buildingsBuilt[df_buildingsBuilt["Type of House"].str.contains("All house types")].loc[:,["Year", "VALUE"]]
#df_buildingsBuilt

,STATISTIC,STATISTIC Label,TLIST(A1),Year,C02541V03076,Component,UNIT,VALUE
0,PEA15,Annual Population Change,1951,1951,1,Annual births,Thousand,NaN
1,PEA15,Annual Population Change,1951,1951,2,Annual deaths,Thousand,NaN
2,PEA15,Annual Population Change,1951,1951,3,Natural increase,Thousand,26.6
3,PEA15,Annual Population Change,1951,1951,4,Immigrants,Thousand,NaN
4,PEA15,Annual Population Change,1951,1951,5,Emigrants,Thousand,NaN
...,...,...,...,...,...,...,...,...
571,PEA15,Annual Population Change,2022,2022,4,Immigrants,Thousand,120.7
572,PEA15,Annual Population Change,2022,2022,5,Emigrants,Thousand,59.6
573,PEA15,Annual Population Change,2022,2022,6,Net migration,Thousand,61.1
574,PEA15,Annual Population Change,2022,2022,7,Population change,Thousand,88.8
